In [1]:
%matplotlib notebook 

import numpy as np 
import matplotlib.pyplot as plt 

import cv2 as cv 
from PIL import Image 

import time

from reachy import Reachy, parts

from pycoral.adapters import classify, common
from pycoral.utils.edgetpu import make_interpreter
from pycoral.utils.dataset import read_label_file

In [2]:
reachy = Reachy(
    head=parts.Head(io='/dev/ttyUSB*'),
)

In [3]:
path_model = '/home/pi/dev/reachy-tictactoe/reachy_tictactoe/models/ttt_classif.tflite'
path_label = '/home/pi/dev/reachy-tictactoe/reachy_tictactoe/models/ttt_classif.txt'

In [ ]:
interpreter = make_interpreter(path_model)
interpreter.allocate_tensors()

labels = read_label_file(path_label)

size = common.input_size(interpreter)

In [33]:
board_cases = np.array((
    ((293, 395, 363, 450),
     (395, 492, 363, 450),
     (492, 590, 363, 450),),

    ((279, 388, 450, 550),
     (388, 498, 450, 550),
     (498, 605, 450, 550),),

    ((258, 379, 550, 675),
     (379, 503, 550, 675),
     (503, 625, 550, 675),),
))

# left, right, top, bottom
board_rect = np.array((
    129, 741, 374, 710,
))

In [6]:
im = reachy.head.get_image()
height, width = np.shape(im)[:2]

In [34]:
video = cv.VideoWriter('tuto_classification.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))

reachy.head.compliant = False 
time.sleep(0.1)

reachy.head.look_at(0.5, 0, -0.34, duration=1, wait=True)

start = time.time()

fps_count = 0

while time.time() - start < 15:
    
    im = reachy.head.get_image()
    out = im.copy()

    for row in range(3):
        for col in range(3):
            lx, rx, ly, ry = board_cases[row, col]
            pil_img = Image.fromarray(out[ly:ry, lx:rx]).convert('RGB').resize(size, Image.ANTIALIAS)
            tic = time.time()
            result = classify.get_classes(interpreter, top_k=1, score_threshold=0.1)
            label = labels.get(res[0].id)
            cv.putText(out, label, (int((lx+rx)/2)-30, int((ly+ry)/2)), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
            cv.rectangle(out, (lx, ly), (rx, ry), (0, 255, 0), 5)
    video.write(out)        
            
video.release()

reachy.head.look_at(0.5, 0, 0, duration=1, wait=True)
time.sleep(0.2)
reachy.head.compliant = True